In [3]:
import numpy as np
import pdb

In [43]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  Xy = np.concatenate((X, y.T, predictions.T), axis=1)
  district_scores = np.array([])
  for key in np.unique(X[:,0]):
    pdb.set_trace()
    district_scores[len(district_scores)] = (
      np.sum(
        (Xy[np.where(Xy[:,0] == key)][:,-2] -
         Xy[np.where(Xy[:,0] == key)][:,-1])/
        Xy[np.where(Xy[:,0] == key)][:,-2]
      ) / num_timeslots
    )
  return np.sum(district_scores) / num_districts

def mape_scorer(estimator, X, y):
  estimator.fit(X)
  predictions = estimator.predict(X)
  return mape(X, predictions, y)

Testing MAPE

In [ ]:
from sklearn.linear_model import LogisticRegression

est = LogisticRegression()
X = np.array([[1, 1], [1, 2], [2, 3], [2, 4]])
predictions = np.array([1, 2, 3, 4])
y = np.array([1, 2, 3, 4])

# Should return 0
mape(X, predictions, y)

> <ipython-input-43-9adc107fc144>(13)mape()
-> np.sum(
(Pdb) Xy[np.where(Xy[:,0] == key)][:,-2]
matrix([[1]])
(Pdb) Xy[np.where(Xy[:,0] == key)][:,-1]
matrix([[1]])
(Pdb) Xy[np.where(Xy[:,0] == key)]
matrix([[1, 1]])
(Pdb) key
1
(Pdb) Xy
matrix([[1, 1, 1, 1],
        [1, 2, 2, 2],
        [2, 3, 3, 3],
        [2, 4, 4, 4]])
(Pdb) Xy[np.where(Xy[:,0] == key)][:]
matrix([[1, 1]])


In [42]:
X[np.where(X[:,0] == 1)]

array([[1, 1],
       [1, 2]])

In [39]:
np.where(X[:, 0] == 2)

(array([2, 3]),)

In [36]:
X

array([[1, 1],
       [1, 2],
       [2, 3],
       [2, 4]])

In [ ]:
features_list = ['']